In [1]:
import os

os.chdir('../')

In [2]:
import os, shutil
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
from typing import Iterable
import torch
from tqdm import tqdm

def _process_one(src_path: Path, dst_path: Path, drop_keys: Iterable[str], legacy_save: bool):
    # 프로세스마다 스레드 과사용 방지
    try:
        torch.set_num_threads(1)
    except Exception:
        pass

    obj = torch.load(src_path, map_location="cpu")
    if isinstance(obj, dict):
        for k in drop_keys:
            obj.pop(k, None)

    tmp = dst_path.with_suffix(dst_path.suffix + ".tmp")
    if legacy_save:
        # 구식 직렬화(대개 쓰기 속도 빠름, 파일 1개로 저장)
        torch.save(obj, tmp, _use_new_zipfile_serialization=False)
    else:
        torch.save(obj, tmp)
    os.replace(tmp, dst_path)  # atomic move
    return dst_path.name

def copy_pt(src_pt_dir: str,
            dst_pt_dir: str,
            drop_keys: Iterable[str] = ("traj", "trajs"),
            max_workers: int = None,
            legacy_save: bool = True,
            skip_existing: bool = True) -> None:
    """
    src_pt_dir의 .pt에서 drop_keys만 제거해 dst_pt_dir로 저장.
    - max_workers: 병렬 처리 프로세스 수 (기본: os.cpu_count()//2 정도 자동)
    - legacy_save: True면 torch.save의 구식 직렬화 사용(보통 더 빠름)
    - skip_existing: True면 대상 파일이 이미 있으면 건너뜀
    """
    src = Path(src_pt_dir); dst = Path(dst_pt_dir)
    dst.mkdir(parents=True, exist_ok=True)

    # config.json 그대로 복사
    if (src / "config.json").exists():
        shutil.copy2(src / "config.json", dst / "config.json")

    def sort_key(p: Path):
        return (0, int(p.stem)) if p.stem.isdigit() else (1, p.stem)

    pt_files = sorted(src.glob("*.pt"), key=sort_key)
    tasks = []
    for f in pt_files:
        out = dst / f.name
        if skip_existing and out.exists():
            continue
        tasks.append((f, out))

    if not tasks:
        return

    if max_workers is None:
        cw = os.cpu_count() or 8
        max_workers = max(1, min(cw // 2, 8))  # I/O라 과도 병렬 비추

    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        futs = [ex.submit(_process_one, f, o, tuple(drop_keys), legacy_save) for f, o in tasks]
        for _ in tqdm(as_completed(futs), total=len(futs), desc="Copying (drop traj)"):
            pass


In [3]:
copy_pt('samplings/dit/train/dit_train_0', 'samplings/dit/train/dit_train_0_trajdrop')

Copying (drop traj): 100%|██████████| 10000/10000 [40:42<00:00,  4.09it/s] 
